In [3]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/srinath/Documents/GitHub/active-learning-text-mining/preprocessed_data_50/preprocessed_Hall.csv')

# Display the first few rows of the DataFrame
df.head()

,label,fault,system,error,use,model,method,propos,test,detect,...,show,measur,implement,oper,current,reliabl,differ,scheme,provid,problem
0,0,0,1,0,2,0,3,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,4,1,0,1,0,0,1,0,0,...,0,0,0,2,0,0,0,0,0,0
2,0,1,3,0,0,2,0,3,0,0,...,0,0,1,0,0,3,0,0,1,0
3,0,0,0,1,0,0,1,2,0,0,...,0,0,2,0,0,0,0,0,0,0
4,0,0,1,0,2,0,0,1,0,1,...,1,0,0,1,0,0,0,0,0,0


In [9]:
wholeY = df['label']
wholeX = df.drop('label', axis=1)

# randomly sample 32 samples with label 1 and 32*4 samples with label 0
# 32 samples with label 1
df1 = df[df['label'] == 1].sample(n=32, random_state=1)
# 32*4 samples with label 0
df0 = df[df['label'] == 0].sample(n=32*6, random_state=1)

# concatenate the two dataframes
traindf = pd.concat([df1, df0])

#split traindf into X and Y
Y = traindf['label']
X = traindf.drop('label', axis=1)

In [10]:
X.shape, Y.shape


((224, 50), (224,))

In [11]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Define the parameter grid
param_grid = {
    'kernel': [1.0 * RBF(length_scale) for length_scale in [1.0, 2.0, 3.0]],
    'max_iter_predict': [100, 200, 300]
}

# Initialize the Gaussian Process Classifier
gpc = GaussianProcessClassifier()

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=gpc, param_grid=param_grid, cv=5, n_jobs=-1, scoring='recall')

# Fit the model
grid_search.fit(X, Y)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Predict using the best estimator
best_gpc = grid_search.best_estimator_
Y_pred = best_gpc.predict(wholeX)

# Print classification report
print(classification_report(wholeY, Y_pred))

/Users/srinath/Documents/GitHub/active-learning-text-mining/venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/srinath/Documents/GitHub/active-learning-text-mining/venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/srinath/Documents/GitHub/active-learning-text-mining/venv/lib/python3.13/site-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing 

Best parameters found:  {'kernel': 1**2 * RBF(length_scale=1), 'max_iter_predict': 100}
Best cross-validation accuracy:  0.719047619047619
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      8807
           1       0.18      0.69      0.28       104

    accuracy                           0.96      8911
   macro avg       0.59      0.83      0.63      8911
weighted avg       0.99      0.96      0.97      8911

